In [1]:
# 更新dask`conda update dask`, 如果出现下面的错误
#   AttributeError: 'module' object has no attribute 'computation'
#   when importing projector
from tensorflow.contrib.tensorboard.plugins import projector
from tqdm import tqdm # progression bars

import numpy as np
import os
import tensorflow as tf

In [2]:
# load一些现成的次向量参数，

# 词向量文件 1: 3.4G, Google's English word2vec embedding parameters
embeddings_path1 = '/your_path_to_embedding_file/GoogleNews-vectors-negative300.bin'

# 词向量文件 2: 3.5G, Facebook's Chinese word2vec embedding parameters
embeddings_path2 = '/your_path_to_embedding_file/fasttext/wiki.zh.bin'

embeddings_path = embeddings_path1
embeddings_format = embeddings_path.split('.')[-1]
print("Loading pre-trained word embeddings from %s, with embdding_format %s"
      % (embeddings_path, embeddings_format))

Loading pre-trained word embeddings from /home/dong/Dropbox/Projects/NLP/DeepQA/data/embeddings/GoogleNews-vectors-negative300.bin, with embdding_format bin


### word2vec, fasttext_embedding等词向量文件包含million量级的词汇，为demo的目的，我们只读前面2000个单词

In [3]:
demo_vocab_size = 2000

# 建立一个 list of vectors 和 a list of words
with open(embeddings_path, "rb") as f:
    header = f.readline()
    vocab_size, vector_size = map(int, header.split())
    binary_len = np.dtype('float32').itemsize * vector_size

    words, embeddings = [], []
    # 每一行是一个单词和一个词向量，使用空格' '分开
    for line in tqdm(range(demo_vocab_size)):
        
        # 读入空格左边的单词, 存入words
        word = []
        while True:
            ch = f.read(1)
            if ch == b' ':
                word = b''.join(word).decode('utf-8')
                break
            word.append(ch)
        words.append(word)
        
        # 读入空格右边的词向量, 存入embeddings
        if embeddings_format == 'bin':
            vector = np.fromstring(f.read(binary_len), dtype='float32')
        elif embeddings_format == 'vec':
            vector = np.fromstring(f.readline(), sep=' ', dtype='float32')
        else:
            raise Exception("Unkown format for embeddings: %s " % embeddings_format)
        embeddings.append(vector)


100%|██████████| 2000/2000 [00:00<00:00, 11818.52it/s]


In [7]:
embeddings[:3]


[array([  1.12915039e-03,  -8.96453857e-04,   3.18527222e-04,
          1.53350830e-03,   1.10626221e-03,  -1.40380859e-03,
         -3.05175781e-05,  -4.19616699e-04,  -5.76019287e-04,
          1.07574463e-03,  -1.02233887e-03,  -6.17980957e-04,
         -7.55310059e-04,   1.40380859e-03,  -1.64031982e-03,
         -6.33239746e-04,   1.63269043e-03,  -1.00708008e-03,
         -1.26647949e-03,   6.52313232e-04,  -4.15802002e-04,
         -1.07574463e-03,   1.52587891e-03,  -2.74658203e-04,
          1.40190125e-04,   1.57165527e-03,   1.35803223e-03,
         -8.31604004e-04,  -1.40380859e-03,   1.57928467e-03,
          2.53677368e-04,  -7.32421875e-04,  -1.05381012e-04,
         -1.16729736e-03,   1.57928467e-03,   6.56127930e-04,
         -6.59942627e-04,   2.92062759e-06,   1.12915039e-03,
          4.27246094e-04,  -3.70025635e-04,  -1.15203857e-03,
          1.26647949e-03,  -3.51667404e-06,   2.65121460e-04,
         -4.02450562e-04,   1.41143799e-04,  -3.36170197e-05,
        

In [8]:
# 开启一个 TensorFlow session,保存2000个词语的embedding matrix参数
# 并保留一个包含词语的 *tsv文件 （用于 tensorboard 中可视化 embedding)

log_path = 'log_0708'

with tf.Session() as sess:
    # tf.assign():这里是一个将具体数值（即，词向量矩阵）赋值给tf Variable的例子：
    X = tf.Variable([0.0], name='embedding')
    place = tf.placeholder(tf.float32, shape=[len(words), vector_size])
    set_x = tf.assign(X, place, validate_shape=False)
    sess.run(tf.global_variables_initializer())
    sess.run(set_x, feed_dict={place: embeddings})

    # 需要保存一个metadata文件,给词典里每一个词分配一个身份
    with open(log_path + '/metadata.tsv', 'w') as f:
        for word in tqdm(words):
            f.write(word + '\n')
    
    # 写 TensorFlow summary
    summary_writer = tf.summary.FileWriter(log_path, sess.graph)
    config = projector.ProjectorConfig()
    embedding_conf = config.embeddings.add()
    embedding_conf.tensor_name = 'embedding:0'
    embedding_conf.metadata_path = os.path.join(log_path, 'metadata.tsv')
    projector.visualize_embeddings(summary_writer, config)
    
    # 保存模型
    # word2vec参数的单词和词向量部分分别保存到了metadata和ckpt文件里面
    saver = tf.train.Saver()
    saver.save(sess, os.path.join(log_path, "model.ckpt"))

100%|██████████| 2000/2000 [00:00<00:00, 1205432.96it/s]


### `tensorboard --logdir log_0708` 打开 tensorboard